<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/PaLM_rlhf_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PaLM + RLHF - Pytorch (ワイプ)**

---
RLHF (Reinforcement Learning with Human Feedback) をPaLMアーキテクチャの上に実装しています。RETROのように検索機能も追加する予定。



# **LoRA(低ランク適応) **

**Low-Rank Adaptation of Large Language Models**


**大規模言語モデルの低ランク適応**

## **インストール**

In [1]:
!pip install palm-rlhf-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 515 kB/s 
     |████████████████████████████████| 191 kB 12.3 MB/s 
     |████████████████████████████████| 702 kB 64.3 MB/s 


# **使用方法**
**初期訓練**
PaLMは、他の自己回帰変換器と同様に

In [2]:
import torch
from palm_rlhf_pytorch import PaLM

palm = PaLM(
    num_tokens = 20000,
    dim = 512,
    depth = 12
).cuda()

seq = torch.randint(0, 20000, (1, 2048)).cuda()

loss = palm(seq, return_loss = True)
loss.backward()

# 多くの訓練の後、順序配列を生成できるようになりました

generated = palm.generate(2048) # (1, 2048)

- そして、**人間の反応**に基づき、**報酬モデル(報酬計算式／報酬計算方法)を学習**させます。
- 元の論文では、**事前学習した変換器**から**報酬モデル(報酬計算式／報酬計算方法)**を**過学習させず**に微調整することはできませんでしたが、まだオープンな研究なので、とにかく**LoRA(Low-Rank Adaptation ,低ランク適応)**で微調整する選択を与えています。

In [3]:
import torch
from palm_rlhf_pytorch import PaLM, RewardModel

palm = PaLM(
    num_tokens = 20000,
    dim = 512,
    depth = 12,
    causal = False
)

reward_model = RewardModel(
    palm,
    num_binned_output = 5 # 1 から 5 までの評価を宣言する
).cuda()

# 疑似データ

seq = torch.randint(0, 20000, (1, 1024)).cuda()
prompt_mask = torch.zeros(1, 1024).bool().cuda() # 順序配列のどの部分がプロンプトで、どの部分が応答であるか
labels = torch.randint(0, 5, (1,)).cuda()

# 訓練

loss = reward_model(seq, prompt_mask = prompt_mask, labels = labels)
loss.backward()

# 多くの訓練の後

reward = reward_model(seq, prompt_mask = prompt_mask)

In [4]:
import torch
from palm_rlhf_pytorch import PaLM, RewardModel, RLHFTrainer

# 事前に訓練されたPaLMをロードする

palm = PaLM(
    num_tokens=256,
    dim=512,
    depth=8
).cuda()

palm.load('./path/to/pretrained/palm.pt')

# 事前訓練済みの報酬モードをロードします

reward_model = RewardModel(
    palm,
    num_binned_output = 5
).cuda()

reward_model.load('./path/to/pretrained/reward_model.pt')

# 強化学習のプロンプトのリストを準備する

prompts = torch.randint(0, 256, (50000, 512)).cuda() # 50k prompts

# すべてをトレーナーに渡してトレーニングする

trainer = RLHFTrainer(
    palm = palm,
    reward_model = reward_model,
    prompt_token_ids = prompts
)

trainer.train(num_episodes = 50000)

# それで成功したら…
# 10個のサンプルを生成し、報酬モデルを使用して最高のものを返します

answer = trainer.generate(2048, prompt = prompts[0], num_samples = 10) # (<= 2048,)

AssertionError: ignored